In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/box/"

In [ ]:
dfa = []
for year in range(2013,2023):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{year}.parquet")
    df1[["Team1","HA","Team2"]] =  df1["MATCHUP"].str.split(" ",expand=True)
    df1["W"] = df1["WL"] == "W"
    df1["L"] = df1["WL"] == "L"
    df2 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{year}_PS.parquet")
    df2[["Team1","HA","Team2"]] =  df2["MATCHUP"].str.split(" ",expand=True)
    df2["W"] = df2["WL"] == "W"
    df2["L"] = df2["WL"] == "L"
    df1g = df1.groupby(["Team1","Team2"])
    df2g = df2.groupby(["Team1","Team2"])
    keys = list(df2g.groups)
    for key in keys:
        df31 = df1g.get_group(key)
        df32 = df2g.get_group(key)
        df4 = pd.DataFrame({"Team1":key[0],"Team2":key[1],"W1":df31["W"].sum(),"L1":df31["L"].sum(),"W2":df32["W"].sum(),"L2":df32["L"].sum(),"MOV1":df31["PLUS_MINUS"].mean(),"MOV2":df32["PLUS_MINUS"].mean()},index=[0])
        df4["Season"] = year + 1
        dfa.append(df4)
df5 = pd.concat(dfa).reset_index(drop=True)

In [ ]:
df5["WP_1"] = round(df5["W1"]/(df5["W1"]+df5["L1"]),3)
df5["WP_2"] = round(df5["W2"]/(df5["W2"]+df5["L2"]),3)
df5["Season"] = df5["Season"].astype("category")

In [ ]:
r = df5["WP_1"].corr(df5["WP_2"])
r2 = r **2
print(r2)

In [ ]:
df5

In [ ]:
p = (
    ggplot(df5, aes(x="WP_1",y="WP_2"))
    + geom_jitter(aes(color="Season"),show_legend=False)
    + geom_smooth(method="lm")
    + scale_x_continuous(labels=percent_format())
    + scale_y_continuous(labels=percent_format())
    + annotate('label', x=0.1, y=1, label=f'r^2={r2:0.3f}', size=14)
    + theme_idv
    + labs(
        title="Regular Season vs Playoffs Matchups",
        subtitle="Jitter plot to reduce overplotting ",
        x="Win Percent: Regular Season",
        y="Win Percent: Playoffs",
        caption="@sradjoker  | source: nba.com/stats via nba_api"
    )
)
p

In [ ]:
r = df5["MOV1"].corr(df5["MOV2"])
r2 = r **2
p = (
    ggplot(df5, aes(x="MOV1",y="MOV2"))
    # + geom_jitter(aes(color="Season"),show_legend=False)
    + geom_point(aes(color="Season"))
    + geom_smooth(method="lm")
    # + scale_x_continuous(labels=percent_format())
    # + scale_y_continuous(labels=percent_format())
    + annotate('label', x=-16, y=20, label=f'r^2={r2:0.3f}', size=14)
    + theme_idv
    + labs(
        title="Regular Season vs Playoffs Matchups",
        # subtitle="Jitter plot to reduce overplotting ",
        x="Avg Margin of Victory: Regular Season",
        y="Avg Margin of Victory: Playoffs",
        caption="@sradjoker  | source: nba.com/stats via nba_api"
    )
)
p